In [1]:
import pandas as pd
import numpy as np

## arrumar casas decimais
pd.set_option('display.float_format', lambda x: '%.5f' % x)

##### Lendo somente as primeiras 5 Linhas do CSV para verificar quais são as colunas

In [2]:
#leitura df completo
df = pd.read_csv(".\DADOS\MICRODADOS_ENEM_2019.csv", encoding='latin-1', sep=';')

##### Verificar dados faltantes

In [3]:
df.isnull().sum().sort_values(ascending=False)

SG_UF_ESC                 3947858
CO_ESCOLA                 3947858
TP_LOCALIZACAO_ESC        3947858
TP_DEPENDENCIA_ADM_ESC    3947858
CO_UF_ESC                 3947858
                           ...   
IN_ACESSO                       0
IN_LEDOR                        0
IN_AMPLIADA_18                  0
IN_AMPLIADA_24                  0
Q025                            0
Length: 136, dtype: int64

In [4]:
total = df.isnull().sum().sort_values(ascending=False)
percentual = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
faltantes = pd.concat([total, percentual], axis=1 , keys=["Total", 'Percentual'])
faltantes

,Total,Percentual
SG_UF_ESC,3947858,77.48084
CO_ESCOLA,3947858,77.48084
TP_LOCALIZACAO_ESC,3947858,77.48084
TP_DEPENDENCIA_ADM_ESC,3947858,77.48084
CO_UF_ESC,3947858,77.48084
...,...,...
IN_ACESSO,0,0.00000
IN_LEDOR,0,0.00000
IN_AMPLIADA_18,0,0.00000
IN_AMPLIADA_24,0,0.00000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5095270 entries, 0 to 5095269
Columns: 136 entries, NU_INSCRICAO to Q025
dtypes: float64(25), int64(70), object(41)
memory usage: 5.2+ GB


#### Tratativa dos dados

In [6]:
colunas = ['SG_UF_RESIDENCIA','NU_IDADE','TP_SEXO','TP_ESTADO_CIVIL','TP_COR_RACA','TP_ST_CONCLUSAO','TP_ANO_CONCLUIU','TP_ESCOLA','IN_TREINEIRO','IN_BAIXA_VISAO','IN_CEGUEIRA','IN_SURDEZ','IN_DEFICIENCIA_AUDITIVA','IN_SURDO_CEGUEIRA','IN_DEFICIENCIA_FISICA','IN_DEFICIENCIA_MENTAL','IN_DEFICIT_ATENCAO','IN_DISLEXIA','IN_DISCALCULIA','IN_AUTISMO','IN_VISAO_MONOCULAR','IN_OUTRA_DEF','IN_GESTANTE','IN_LACTANTE','IN_SEM_RECURSO','TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','TP_LINGUA','TP_STATUS_REDACAO','NU_NOTA_REDACAO','Q001','Q002','Q006','Q024','Q025']
df_novo = df.loc[:, colunas]

#Coluna de qtd. deficientes
df_novo['DEF'] = np.where(df_novo.iloc[:, 10:24].sum(axis=1) > 0,1,0)

#Colune Nota média dos participantes
df_novo['NOTA_MEDIA'] = (df_novo.loc[:,('NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_CN', 'NU_NOTA_REDACAO')].sum(axis=1) / 5)
df_novo['Q024'] = np.where(df_novo.loc[:,'Q024'] == 'A',0,1)
df_novo['Q025'] = np.where(df_novo.loc[:,'Q025'] == 'A',0,1)
df_novo["Q024"] = df_novo['Q024'].apply(np.sign).replace({ 0:'Não tem computador', 1: 'Tem computador'})
df_novo["Q025"] = df_novo['Q025'].apply(np.sign).replace({ 0:'Não tem internet', 1: 'Tem internet'})

##quantidade de pessoas presentes em todas as provas
df_novo['PRESENCA'] = np.where(df_novo.iloc[:, 26:29].sum(axis=1) > 0,1,0)

#drop nas colunas de deficientes
df_novo.drop(df_novo.columns[10:24],axis=1,inplace=True)



In [14]:
df_novo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3701947 entries, 3 to 5095264
Data columns (total 30 columns):
 #   Column             Dtype  
---  ------             -----  
 0   SG_UF_RESIDENCIA   object 
 1   NU_IDADE           float64
 2   TP_SEXO            object 
 3   TP_ESTADO_CIVIL    int64  
 4   TP_COR_RACA        int64  
 5   TP_ST_CONCLUSAO    int64  
 6   TP_ANO_CONCLUIU    int64  
 7   TP_ESCOLA          int64  
 8   IN_TREINEIRO       int64  
 9   IN_BAIXA_VISAO     int64  
 10  IN_SEM_RECURSO     int64  
 11  TP_PRESENCA_CN     int64  
 12  TP_PRESENCA_CH     int64  
 13  TP_PRESENCA_LC     int64  
 14  TP_PRESENCA_MT     int64  
 15  NU_NOTA_CN         float64
 16  NU_NOTA_CH         float64
 17  NU_NOTA_LC         float64
 18  NU_NOTA_MT         float64
 19  TP_LINGUA          int64  
 20  TP_STATUS_REDACAO  float64
 21  NU_NOTA_REDACAO    float64
 22  Q001               object 
 23  Q002               object 
 24  Q006               object 
 25  Q024              

In [ ]:
df_novo.shape

In [ ]:
df_novo.describe()


In [ ]:
df_novo.describe(include='O')

In [ ]:
# Pessoas com nota máxima na redação
len(df_novo[df_novo['NU_NOTA_REDACAO'] == 1000.0])

In [ ]:
# Participantes que zeraram na prova
len(df_novo[df_novo['NOTA_MEDIA'] == 0.00])

In [ ]:
df_sorted = df_novo.sort_values('NOTA_MEDIA', ascending=False)
df_sorted

In [ ]:
#drop notas em branco
df_novo.dropna(subset=['NOTA_MEDIA'],inplace=True)

In [ ]:
generoXmedia=pd.pivot_table(df_novo,values=['NOTA_MEDIA'],columns=["TP_SEXO"])
estadoXmedia=pd.pivot_table(df_novo,values=['NOTA_MEDIA'],columns=['SG_UF_RESIDENCIA'])
defXmedia=pd.pivot_table(df_novo,values=['NOTA_MEDIA'],columns=["DEF"])
computXinternetXmedia=pd.pivot_table(df_novo,values=['NOTA_MEDIA'],columns=["Q024","Q025"])

In [ ]:
print(generoXmedia)
print(estadoXmedia)
print(defXmedia)
print(escolarXmedia)
print(computXinternetXmedia)

In [ ]:
#Aqui substitui os estados por regiões para melhor visualização
df_novo["SG_UF_RESIDENCIA"]=df_novo.filter(items=["SG_UF_RESIDENCIA"]).replace(['AM','PA','AC','RO','RR','AP','TO'],"NORTE")
df_novo["SG_UF_RESIDENCIA"]=df_novo.filter(items=["SG_UF_RESIDENCIA"]).replace(['MA','PI','CE','RN','PB','PE','AL','SE','BA'],"NORDESTE")
df_novo["SG_UF_RESIDENCIA"]=df_novo.filter(items=["SG_UF_RESIDENCIA"]).replace(['MT','MS','GO','DF'],"CENTRO-OESTE")
df_novo["SG_UF_RESIDENCIA"]=df_novo.filter(items=["SG_UF_RESIDENCIA"]).replace(['MG','SP','RJ','ES'],"SUDESTE")
df_novo["SG_UF_RESIDENCIA"]=df_novo.filter(items=["SG_UF_RESIDENCIA"]).replace(['PR','SC','RS'],"SUL")
regiaoXmedia=pd.pivot_table(df_novo,values=['NOTA_MEDIA'],columns=['SG_UF_RESIDENCIA'])

In [ ]:
print(regiaoXmedia)

##### Selecionando os melhores 1000 participantes (TOP 1000)

In [ ]:
df_top1k = df_novo.NOTA_MEDIA.nlargest(1000)
df_top1k

In [ ]:
nota_corte = df_top1k.min()
nota_corte

In [ ]:
df_top1k = df_sorted[df_sorted['NOTA_MEDIA'] >= nota_corte]

In [ ]:
df_top1k.reset_index(drop=True)

----------------------

## COMPARANDO PARTICIPANTES NO TOTAL E NOS 1000 MELHORES COLOCADOS

In [ ]:
df_top1k.replace({0:'Não declarado', 1:'Branca',2:'Preta',3:'Parda',4:'Amarela',5:'Indígena'},inplace=True)

In [ ]:
df_sorted.TP_COR_RACA.value_counts()

In [ ]:
df_sorted.TP_COR_RACA.value_counts(normalize=True)*100

In [ ]:
df_top1k.TP_COR_RACA.value_counts()

In [ ]:
df_top1k.TP_COR_RACA.value_counts(normalize=True)*100

In [ ]:
df_sorted.TP_SEXO.value_counts(normalize=True)*100

In [ ]:
df_top1k.TP_SEXO.value_counts(normalize=True)*100

In [ ]:
df_sorted.Q006.value_counts(normalize=True)*100

In [ ]:
df_top1k.Q006.value_counts(normalize=True)*100

In [ ]:
df_top1k.SG_UF_RESIDENCIA.value_counts(normalize=True)*100

### Analise de Presentes e Ausentes

In [ ]:
#Analise de presentes e ausentes totais por estado
df_novo.groupby(['SG_UF_RESIDENCIA']).agg({'PRESENCA':np.sum}).sort_values(by='PRESENCA', ascending=True)

In [ ]:
#Idade média por presenca sexo e tipo de escola
df_novo.groupby(['PRESENCA','TP_SEXO','TP_ESCOLA']).agg({'NU_IDADE':np.mean})

In [ ]:
#Data Frame sem os ausentes em todas as provas
df_sem_ausentes= df_novo[df_novo['PRESENCA'] != 0]

In [ ]:
#Estados com o menor e maior numero de participantes presentes
df_novo.groupby(by="SG_UF_RESIDENCIA").size().sort_values()

### Analise de Gestantes, Idosos, Candidatos

In [7]:
df_novo = df_novo.dropna()
idades = df['NU_IDADE']
gestantes = df['IN_GESTANTE']
sexos = df['TP_SEXO']

In [8]:
# quantidade e distribuição por idade de candidatos menores de 18 anos
print(f'Um total de {idades[idades < 18].count()} candidatos menores de 18 anos realizaram o exame')
print(f'Distribuição dos {idades[idades < 18].count()} menores por idade:')
print(idades[idades < 18].value_counts().sort_index())

Um total de 1247897 candidatos menores de 18 anos realizaram o exame
Distribuição dos 1247897 menores por idade:
10.00000         6
11.00000         9
12.00000        46
13.00000       266
14.00000      5267
15.00000     78563
16.00000    316437
17.00000    847303
Name: NU_IDADE, dtype: int64


In [9]:
# notas de acordo com a escolaridade do pai
display(df_novo.filter(items=['NU_NOTA_REDACAO', 'Q001']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q001').mean().sort_values(by='NU_NOTA_REDACAO', ascending = False))
display(df_novo.filter(items=['NU_NOTA_MT', 'Q001']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q001').mean().sort_values(by='NU_NOTA_MT', ascending = False))
display(df_novo.filter(items=['NU_NOTA_LC', 'Q001']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q001').mean().sort_values(by='NU_NOTA_LC', ascending = False))
display(df_novo.filter(items=['NU_NOTA_CH', 'Q001']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q001').mean().sort_values(by='NU_NOTA_CH', ascending = False))
display(df_novo.filter(items=['NU_NOTA_CN', 'Q001']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q001').mean().sort_values(by='NU_NOTA_CN', ascending = False))

,NU_NOTA_REDACAO
Q001,
Completou a Pós-graduação,700.47457
"Completou a Faculdade, mas não completou a Pós-graduação",673.47412
"Completou o Ensino Médio, mas não completou a Faculdade",604.81116
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",574.55501
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",558.37145
Não sabe,537.06127
Não completou a 4ª série/5º ano do Ensino Fundamental,532.60373
Nunca estudou,496.08196


,NU_NOTA_MT
Q001,
Completou a Pós-graduação,624.11831
"Completou a Faculdade, mas não completou a Pós-graduação",600.30673
"Completou o Ensino Médio, mas não completou a Faculdade",538.92461
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",515.74746
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",505.39859
Não sabe,495.47829
Não completou a 4ª série/5º ano do Ensino Fundamental,486.45647
Nunca estudou,466.09819


,NU_NOTA_LC
Q001,
Completou a Pós-graduação,569.34135
"Completou a Faculdade, mas não completou a Pós-graduação",560.80198
"Completou o Ensino Médio, mas não completou a Faculdade",533.24510
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",519.75659
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",513.20343
Não sabe,509.17439
Não completou a 4ª série/5º ano do Ensino Fundamental,502.12592
Nunca estudou,486.66172


,NU_NOTA_CH
Q001,
Completou a Pós-graduação,572.56784
"Completou a Faculdade, mas não completou a Pós-graduação",559.57319
"Completou o Ensino Médio, mas não completou a Faculdade",521.15866
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",504.56876
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",497.79445
Não sabe,492.36669
Não completou a 4ª série/5º ano do Ensino Fundamental,486.21110
Nunca estudou,471.99437


,NU_NOTA_CN
Q001,
Completou a Pós-graduação,543.66565
"Completou a Faculdade, mas não completou a Pós-graduação",528.93054
"Completou o Ensino Médio, mas não completou a Faculdade",488.24876
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",471.94079
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",465.23487
Não sabe,459.85927
Não completou a 4ª série/5º ano do Ensino Fundamental,454.41733
Nunca estudou,442.31117


In [10]:
# notas de acordo com a escolaridade da mãe
display(df_novo.filter(items=['NU_NOTA_REDACAO', 'Q002']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q002').mean().sort_values(by='NU_NOTA_REDACAO', ascending = False))
display(df_novo.filter(items=['NU_NOTA_MT', 'Q002']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q002').mean().sort_values(by='NU_NOTA_MT', ascending = False))
display(df_novo.filter(items=['NU_NOTA_LC', 'Q002']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q002').mean().sort_values(by='NU_NOTA_LC', ascending = False))
display(df_novo.filter(items=['NU_NOTA_CH', 'Q002']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q002').mean().sort_values(by='NU_NOTA_CH', ascending = False))
display(df_novo.filter(items=['NU_NOTA_CN', 'Q002']).replace({'A': 'Nunca estudou', 'B': 'Não completou a 4ª série/5º ano do Ensino Fundamental', 'C': 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental', 'D': 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio', 'E': 'Completou o Ensino Médio, mas não completou a Faculdade', 'F': 'Completou a Faculdade, mas não completou a Pós-graduação', 'G': 'Completou a Pós-graduação', 'H': 'Não sabe'}).groupby('Q002').mean().sort_values(by='NU_NOTA_CN', ascending = False))

,NU_NOTA_REDACAO
Q002,
Completou a Pós-graduação,676.23782
"Completou a Faculdade, mas não completou a Pós-graduação",655.62512
"Completou o Ensino Médio, mas não completou a Faculdade",591.13498
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",558.91717
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",543.68486
Não completou a 4ª série/5º ano do Ensino Fundamental,522.13262
Não sabe,505.26553
Nunca estudou,482.74314


,NU_NOTA_MT
Q002,
Completou a Pós-graduação,598.91668
"Completou a Faculdade, mas não completou a Pós-graduação",583.32451
"Completou o Ensino Médio, mas não completou a Faculdade",528.31758
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",505.04633
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",496.05030
Não sabe,484.99646
Não completou a 4ª série/5º ano do Ensino Fundamental,480.07189
Nunca estudou,459.85939


,NU_NOTA_LC
Q002,
Completou a Pós-graduação,558.16144
"Completou a Faculdade, mas não completou a Pós-graduação",552.29794
"Completou o Ensino Médio, mas não completou a Faculdade",527.57132
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",514.19840
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",508.28169
Não completou a 4ª série/5º ano do Ensino Fundamental,499.02568
Não sabe,495.27103
Nunca estudou,483.94099


,NU_NOTA_CH
Q002,
Completou a Pós-graduação,556.85678
"Completou a Faculdade, mas não completou a Pós-graduação",547.87312
"Completou o Ensino Médio, mas não completou a Faculdade",513.77736
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",497.69051
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",492.42522
Não completou a 4ª série/5º ano do Ensino Fundamental,483.59152
Não sabe,479.13700
Nunca estudou,471.00762


,NU_NOTA_CN
Q002,
Completou a Pós-graduação,527.19592
"Completou a Faculdade, mas não completou a Pós-graduação",517.18657
"Completou o Ensino Médio, mas não completou a Faculdade",480.87176
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio",464.92331
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental",459.65398
Não completou a 4ª série/5º ano do Ensino Fundamental,451.43099
Não sabe,450.92090
Nunca estudou,440.76825


In [11]:
# percentual de mulheres gestantes
numGestantes = gestantes.value_counts()
percentGestantes = [100*x/numGestantes.sum() for x in numGestantes]
numSexos = sexos.value_counts()
percentSexos = [100*x/numSexos.sum() for x in numSexos]
numSexoFeminino = numSexos[0]
numGestantes = numGestantes[1]
percentGestantes = 100*numGestantes/numSexoFeminino
print(f'Percentual de mulheres gestantes {percentGestantes:.3f}%')

Percentual de mulheres gestantes 0.047%


In [12]:
# quantidade de candidatos homens
numSexoMasculino = numSexos[1]
print(f'A quantidade de candidatos autodeclarados como homens é de {numSexoMasculino}')

print('-='*50)

# quantidade de candidatas mulheres
print(f'A quantidade de candidatas autodeclarados como mulheres é de {numSexoFeminino}')

A quantidade de candidatos autodeclarados como homens é de 2063448
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
A quantidade de candidatas autodeclarados como mulheres é de 3031822


In [13]:
# quantidade e distribuição por idade de candidatos idosos
print(f'Um total de {idades[idades >= 60].count()} idosos fizeram o Enem em 2019, nunca é tarde para se perseguir metas!')
print(f'Distribuição dos {idades[idades >= 60].count()} idosos por idade:')
print(idades[idades >= 60].value_counts().sort_index())

Um total de 10160 idosos fizeram o Enem em 2019, nunca é tarde para se perseguir metas!
Distribuição dos 10160 idosos por idade:
60.00000    1901
61.00000    1602
62.00000    1384
63.00000    1075
64.00000     849
65.00000     713
66.00000     531
67.00000     495
68.00000     366
69.00000     306
70.00000     221
71.00000     174
72.00000     127
73.00000     101
74.00000      78
75.00000      58
76.00000      41
77.00000      34
78.00000      31
79.00000      20
80.00000      11
81.00000      14
82.00000       8
83.00000       6
84.00000       3
85.00000       2
86.00000       3
87.00000       1
88.00000       1
91.00000       2
92.00000       1
94.00000       1
Name: NU_IDADE, dtype: int64
